Combine meteorological data (HRRR) with crop yield data (USDA) to use in simple regression neural net.

**NOTE**: change ``GOOGLE_DRIVE_PATH_POST_MYDRIVE`` to match your system

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_POST_MYDRIVE = 'DL/MMST-ViT-main'  # TODO change this
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['models_pvt.py', 'main_finetune_mmst_vit.py', 'models_mmst_vit.py', 'attention.py', 'README.md', 'LICENSE', 'requirements.txt', 'models_pvt_simclr.py', 'data', 'input', 'util', 'dataset', 'loss', 'config', '__pycache__', 'output_dir', 'main_pretrain_mmst_vit.py', 'gsheet2csv.ipynb', 'train_debug.ipynb', 'GridSearchSandbox.ipynb', 'mmst_vit_output', 'mmst_vit_logs', 'simple_model', "Abhishek's training.ipynb", 'mmst_vit_output_bs16_lr0.0001_wd0.01_we10', 'mmst_vit_logs_bs16_lr0.0001_wd0.01_we10', 'mmst_vit_output_bs16_lr0.0001_wd0.05_we10', 'mmst_vit_logs_bs16_lr0.0001_wd0.05_we10']


In [ ]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH)

if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

Running in google colab. Our path is `/content/drive/MyDrive/DL/MMST-ViT-main`


In [ ]:
import random
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Pytorch package
import torch


In [ ]:
# Get USDA data for each crop
# starting with just cotton
crop = 'WinterWheat' # OPTIONS: Corn, Cotton, Soybeans, WinterWheat
usda_data_path = 'data/USDA/data'
data_path = os.path.join(GOOGLE_DRIVE_PATH, usda_data_path, crop)
data_list = []
for filename in glob.iglob(data_path + "/**", recursive=True):
  if os.path.isfile(filename) and filename[-3:] == 'csv': # filter dirs and non-csv files
    if crop == 'Soybeans' and (f'{crop}_County' not in filename or '(1)' in filename):
      # print(f"Skipping file: {filename}")
      continue
    else:
      # print(filename)
      df = pd.read_csv(filename)
      data_list.append(df)

usda = pd.concat(data_list)
print(usda['state_name'].unique())
# print(usda.describe())

['ARKANSAS' 'CALIFORNIA' 'COLORADO' 'IDAHO' 'ILLINOIS' 'INDIANA' 'KANSAS'
 'KENTUCKY' 'MARYLAND' 'MICHIGAN' 'MISSISSIPPI' 'MISSOURI' 'MONTANA'
 'NEBRASKA' 'NORTH CAROLINA' 'NORTH DAKOTA' 'OHIO' 'OKLAHOMA' 'OREGON'
 'SOUTH DAKOTA' 'TENNESSEE' 'TEXAS' 'VIRGINIA' 'WASHINGTON' 'WISCONSIN']


In [ ]:
# select key columns only - potentially use asd_desc as dummy variable?
yield_var = 'YIELD, MEASURED IN BU / ACRE'
if crop == 'Cotton':
  yield_var = 'YIELD, MEASURED IN LB / ACRE'
usda = usda[['state_name', 'county_name', 'year', yield_var,
           'commodity_desc', 'asd_desc']]
usda = usda.rename(columns={yield_var: 'yield', "state_name": "state", "county_name": "county"})
# print(usda.describe()
print(f"{usda.columns=}")
print(f'{usda.isnull().sum().sum()=}')

usda.columns=Index(['state', 'county', 'year', 'yield', 'commodity_desc', 'asd_desc'], dtype='object')
usda.isnull().sum().sum()=0


In [ ]:
# combine all monthly HRRR data into 1 data frame with 1 row for each state/county/year
# NOTE: there are a number of files- this will take a few minutes
start_year = 2017
end_year = 2022
monthly_data = []
state_list = ['IA', 'IL', 'LA', 'MS']
join_cols = ['State', 'County', 'Year']
for state_abbr in state_list:
  for year in range(start_year, end_year + 1):
    df_monthly = pd.DataFrame()
    print(f"{crop=} {state_abbr=} {year=}")
    data_path = os.path.join(GOOGLE_DRIVE_PATH, f'data/HRRR/data/{year}/{state_abbr}')
    file_list = glob.glob(f"{data_path}/*.csv")
    file_list.sort()
    for filename in file_list:
        # print(filename)
        df = pd.read_csv(filename)
        df_month_avg = df[df['Daily/Monthly'] == 'Monthly']
        month = df_month_avg['Month'].unique()[0]
        df_month_avg = df_month_avg.drop(
            columns=['Month', 'Day', 'Daily/Monthly', 'FIPS Code', 'Grid Index',
                     'Lat (llcrnr)', 'Lon (llcrnr)', 'Lat (urcrnr)','Lon (urcrnr)',
                     'U Component of Wind (m s**-1)', 'V Component of Wind (m s**-1)'])
        if month==1:
          df_monthly = df_month_avg
        else:
          df_monthly = df_monthly.merge(df_month_avg, how='outer', on=join_cols,
                                        suffixes=(None, f"_{month}"))
    monthly_data.append(df_monthly)

print('\nFinished!')

crop='WinterWheat' state_abbr='IA' year=2017
crop='WinterWheat' state_abbr='IA' year=2018
crop='WinterWheat' state_abbr='IA' year=2019
crop='WinterWheat' state_abbr='IA' year=2020
crop='WinterWheat' state_abbr='IA' year=2021
crop='WinterWheat' state_abbr='IA' year=2022
crop='WinterWheat' state_abbr='IL' year=2017
crop='WinterWheat' state_abbr='IL' year=2018
crop='WinterWheat' state_abbr='IL' year=2019
crop='WinterWheat' state_abbr='IL' year=2020
crop='WinterWheat' state_abbr='IL' year=2021
crop='WinterWheat' state_abbr='IL' year=2022
crop='WinterWheat' state_abbr='LA' year=2017
crop='WinterWheat' state_abbr='LA' year=2018
crop='WinterWheat' state_abbr='LA' year=2019
crop='WinterWheat' state_abbr='LA' year=2020
crop='WinterWheat' state_abbr='LA' year=2021
crop='WinterWheat' state_abbr='LA' year=2022
crop='WinterWheat' state_abbr='MS' year=2017
crop='WinterWheat' state_abbr='MS' year=2018
crop='WinterWheat' state_abbr='MS' year=2019
crop='WinterWheat' state_abbr='MS' year=2020
crop='Wint

In [ ]:
# check results
print(f"{len(monthly_data)=}")
print(monthly_data[0].columns[0:20])
print(f'{monthly_data[0].isnull().sum().sum()=}')
monthly_data[0]


len(monthly_data)=24
Index(['Year', 'State', 'County', 'Avg Temperature (K)', 'Max Temperature (K)',
       'Min Temperature (K)', 'Precipitation (kg m**-2)',
       'Relative Humidity (%)', 'Wind Gust (m s**-1)', 'Wind Speed (m s**-1)',
       'Downward Shortwave Radiation Flux (W m**-2)',
       'Vapor Pressure Deficit (kPa)', 'Avg Temperature (K)_2',
       'Max Temperature (K)_2', 'Min Temperature (K)_2',
       'Precipitation (kg m**-2)_2', 'Relative Humidity (%)_2',
       'Wind Gust (m s**-1)_2', 'Wind Speed (m s**-1)_2',
       'Downward Shortwave Radiation Flux (W m**-2)_2'],
      dtype='object')
monthly_data[0].isnull().sum().sum()=0


,Year,State,County,Avg Temperature (K),Max Temperature (K),Min Temperature (K),Precipitation (kg m**-2),Relative Humidity (%),Wind Gust (m s**-1),Wind Speed (m s**-1),...,Vapor Pressure Deficit (kPa)_11,Avg Temperature (K)_12,Max Temperature (K)_12,Min Temperature (K)_12,Precipitation (kg m**-2)_12,Relative Humidity (%)_12,Wind Gust (m s**-1)_12,Wind Speed (m s**-1)_12,Downward Shortwave Radiation Flux (W m**-2)_12,Vapor Pressure Deficit (kPa)_12
0,2017,IOWA,ADAIR,270.201664,273.797581,267.534022,1.457522,82.379301,6.873226,4.071863,...,0.289358,269.669573,274.877997,265.374745,0.812960,68.327688,7.703694,4.474161,1786.745699,0.202336
1,2017,IOWA,ADAMS,270.292339,274.352073,267.355476,1.514278,81.846371,6.427810,3.812052,...,0.308500,270.204435,275.569698,265.824879,0.770476,67.845565,7.107073,4.150411,1792.953226,0.212589
2,2017,IOWA,ALLAMAKEE,267.865630,270.680827,265.263846,3.617301,81.920883,5.723956,3.377784,...,0.210979,267.865844,272.322202,264.379691,1.023985,71.878098,6.624326,3.696295,1361.897284,0.139781
3,2017,IOWA,APPANOOSE,271.054653,275.019294,268.095911,2.129198,80.416129,6.976698,4.101601,...,0.291629,270.804097,276.095839,266.726440,0.644565,66.966935,7.470423,4.350685,1717.462500,0.216371
4,2017,IOWA,AUDUBON,269.428165,273.045047,266.639921,2.002703,84.115054,6.955459,4.072785,...,0.277367,269.117921,274.135025,264.738842,0.688943,69.047312,7.641190,4.440513,1857.069534,0.190964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2017,IOWA,WINNEBAGO,267.026827,270.206552,264.240911,2.662698,85.054435,7.550718,4.734306,...,0.181696,266.776012,271.401919,262.907407,0.937669,75.272177,8.080214,4.764673,1415.393145,0.122879
95,2017,IOWA,WINNESHIEK,267.232413,270.100042,264.673891,3.230625,83.702823,6.487200,3.944087,...,0.182146,267.179921,271.494153,263.845478,0.898992,73.100403,6.992643,3.924032,1351.585081,0.125593
96,2017,IOWA,WOODBURY,268.748221,272.503024,265.828347,1.900955,82.066501,7.007804,4.087562,...,0.261713,268.853304,273.965119,264.461388,1.132932,70.578660,7.091440,4.119439,1548.728908,0.169826
97,2017,IOWA,WORTH,266.820960,269.936927,264.133516,2.734859,84.778226,7.227036,4.524129,...,0.181637,266.738137,271.259581,262.781548,0.839246,75.044355,7.834190,4.544060,1386.084677,0.123968


In [ ]:
# gather each state/county/year into 1 dataframe
monthly_hrrr = pd.concat(monthly_data)
# monthly_hrrr.to_csv('/content/drive/My Drive/DL/DL_project/monthly_hrrr_debug.csv', index=False)

print(monthly_hrrr.head(3))
print(f'{monthly_hrrr.isnull().sum().sum()=}')



   Year State     County  Avg Temperature (K)  Max Temperature (K)  \
0  2017  IOWA      ADAIR           270.201664           273.797581   
1  2017  IOWA      ADAMS           270.292339           274.352073   
2  2017  IOWA  ALLAMAKEE           267.865630           270.680827   

   Min Temperature (K)  Precipitation (kg m**-2)  Relative Humidity (%)  \
0           267.534022                  1.457522              82.379301   
1           267.355476                  1.514278              81.846371   
2           265.263846                  3.617301              81.920883   

   Wind Gust (m s**-1)  Wind Speed (m s**-1)  ...  \
0             6.873226              4.071863  ...   
1             6.427810              3.812052  ...   
2             5.723956              3.377784  ...   

   Vapor Pressure Deficit (kPa)_11  Avg Temperature (K)_12  \
0                         0.289358              269.669573   
1                         0.308500              270.204435   
2                  

In [ ]:
# inner join HRRR with USDA data (on fips or state, county, fips - check match)
monthly_hrrr = monthly_hrrr.rename(columns={'State': 'state', 'County': 'county', 'Year': 'year'})
# print(f"{monthly_hrrr['state'].unique()=}")
# print(f"{usda['state'].unique()= }")

combo = monthly_hrrr.merge(usda, on=['state', 'county', 'year',], how='inner')
print(f"{combo['state'].unique()= }")
print(f'{len(combo)=}, {len(monthly_hrrr)=}, {len(usda)=}')
print(f'{combo.isnull().sum().sum()=}')


combo['state'].unique()= array(['ILLINOIS', 'MISSISSIPPI'], dtype=object)
len(combo)=398, len(monthly_hrrr)=2082, len(usda)=5082
combo.isnull().sum().sum()=0


In [ ]:
# save as csv
combo.to_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}.csv', index=False)

# save another version with only parameters to train model
model_data = combo.drop(columns=['year', 'state', 'county', 'commodity_desc', 'asd_desc' ])
model_data.to_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}_model_data.csv', index=False)


Create combined dataframe with all crops + area description as dummy variables

In [ ]:
# Join all 4 crops
# NOTE: only run after created combined monthly csv files for each crop
corn = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_Corn.csv')
cotton = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_Cotton.csv')
soybeans = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_Soybeans.csv')
winterwheat = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_WinterWheat.csv')
print(f'{len(corn)=}, {len(cotton)=}, {len(soybeans)=}, {len(winterwheat)=}')

len(corn)=1386, len(cotton)=231, len(soybeans)=1413, len(winterwheat)=398


In [ ]:
combo = pd.concat([corn, cotton, soybeans, winterwheat])
print(f'{len(combo)=}')
combo  = combo.drop(columns=['year', 'state', 'county'])
combo = pd.get_dummies(combo, columns=['commodity_desc', 'asd_desc'], drop_first=True)
print(f"New commodity (crop) columns: {[col for col in combo if col.startswith('commodity_desc')]}")
combo.to_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/all_crops.csv', index=False)

len(combo)=3428
New commodity (crop) columns: ['commodity_desc_COTTON', 'commodity_desc_SOYBEANS', 'commodity_desc_WHEAT']


In [ ]:
# add region variable to each crop individually
crops = ['Corn', 'Cotton', 'Soybeans', 'WinterWheat']
for crop in crops:
  df = pd.read_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}.csv')
  df = df.drop(columns=['year', 'state', 'county', 'commodity_desc'])
  df = pd.get_dummies(df, columns=['asd_desc'], drop_first=True)
  df.to_csv(f'{GOOGLE_DRIVE_PATH}/simple_model/monthly_{crop}_with_region.csv', index=False)
  print(f'Created {crop} csv with region dummy cols')

Created Corn csv with region dummy cols
Created Cotton csv with region dummy cols
Created Soybeans csv with region dummy cols
Created WinterWheat csv with region dummy cols


BELOW THIS IS OLD CODE (alternate version kept in case want to re-use them later)
___________________________________________


In [ ]:
# # explore why so few => cotton has fewer states (not Iowa)
# df1 = monthly_hrrr[["state", "county", "year","Avg Temperature (K)" ]]
# df2 = usda[["state", "county", "year", "yield"]]
# adair_yields = usda[usda['state'] == 'IOWA']
# print(adair_yields.head())

# # # antijoin
# # outer = df1.merge(df2, how='outer', indicator=True)

# # anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
# # print(anti_join.head())

In [ ]:
# # combine all HRRR data into 1 data frame
# # NOTE: there are a number of files- this will take a few minutes
# data_path = os.path.join(GOOGLE_DRIVE_PATH, 'data/HRRR/data')
# data_list = []
# for filename in glob.iglob(data_path + "/**", recursive=True):
#   if os.path.isfile(filename) and filename[-3:] == 'csv': # filter dirs and non-csv files
#       print(filename)
#       df = pd.read_csv(filename)
#       data_list.append(df)
# hrrr = pd.concat(data_list)
# print('Finished!')

In [ ]:
# # print(hrrr.describe())
# print(f"{hrrr.columns=}")
# print(f'{hrrr.isnull().sum().sum()=}')

In [ ]:
# # split into daily vs monthly data sets
# hrrr_daily = hrrr[hrrr['Daily/Monthly'] == 'Daily']
# hrrr_monthly = hrrr[hrrr['Daily/Monthly'] == 'Monthly']
# print(f'{len(hrrr_daily)=}, {len(hrrr_monthly)=}')

In [ ]:
# # rename  + select key columns
# hrrr_monthly = hrrr_monthly.drop(columns=['Day', 'Daily/Monthly',
#        'FIPS Code', 'Grid Index', 'Lat (llcrnr)', 'Lon (llcrnr)', 'Lat (urcrnr)',
#        'Lon (urcrnr)'])
# hrrr_monthly = hrrr_monthly.rename(columns={'Year': 'year', 'Month':'month', "State": "state", "County": "county"})
# print(f"{hrrr_monthly.columns=}")
# print(f'{hrrr_monthly.isnull().sum().sum()=}')

In [ ]:
# # reshape monthly hrrr data  to have 1 row per year/county/state
# # first check column
# #print(hrrr_monthly.sort_values(by = ['county', 'year', 'month']).head(3))
# df_wide = hrrr_monthly.pivot_table(index=['state', 'county', 'year'], columns='month', values=['Avg Temperature (K)',
#        'Max Temperature (K)', 'Min Temperature (K)',
#        'Precipitation (kg m**-2)', 'Relative Humidity (%)',
#        'Wind Gust (m s**-1)', 'Wind Speed (m s**-1)',
#        'U Component of Wind (m s**-1)', 'V Component of Wind (m s**-1)',
#        'Downward Shortwave Radiation Flux (W m**-2)',
#        'Vapor Pressure Deficit (kPa)'])
# print(df_wide)


In [ ]:
# # join USDA with other csv to get fips codes - NOT NEEDED?
# fips_file = os.path.join(GOOGLE_DRIVE_PATH, 'input', 'county_info.csv')
# fips = pd.read_csv(fips_file)
# fips = fips.rename(columns={"FIPS Code": "fips", "State": "state_name", "County": "county_name"})
# print(fips.columns)
# df = usda.merge(fips, on=['state_name', 'county_name'], how='inner')
# print(f'{len(usda)=}, {len(df)=}')

# # antijoin
# outer = usda.merge(fips, how='outer', indicator=True)

# anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
# print(anti_join.head())
